In [1]:
# Import and setup
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
import matplotlib.pyplot as plt
import gc
%matplotlib inline

In [2]:
# Read training data
df_train = pd.read_csv('train_V2.csv', nrows=None)

# Drop NA
df_train = df_train.dropna()
print(df_train.shape)
df_train.head()

(4446965, 29)


,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,1241,0,0,0.00,1306,squad-fpp,28,26,-1,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,0,0,0,0.00,1777,squad-fpp,26,25,1484,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,0,0,0,0.00,1318,duo,50,47,1491,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,0,0,0,0.00,1436,squad-fpp,31,30,1408,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,0,1,1,58.53,1424,solo-fpp,97,95,1560,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875


In [3]:
# Define class to preprocess raw data
class DfTransformer:
    def __init__(self):
        self.mean_rankpoints = None
        self.mean_killpoints = None
        self.mean_winpoints = None
        self.df_test_meta = None
    
    def transform_train(self, df):
        # Replace None values of rankPoints with mean
        col_rankpoints = df['rankPoints']
        col_rankpoints = col_rankpoints.loc[col_rankpoints > 1e-4]
        self.mean_rankpoints = col_rankpoints.mean()
        # print('The mean of non-none rankPoints is %.3f' % mean_rankpoints)
        df.loc[df['rankPoints'] < 1e-4, 'rankPoints'] = self.mean_rankpoints
        
        # Replace None values of killPoints with mean
        col_killpoints = df['killPoints']
        col_killpoints = col_killpoints.loc[col_killpoints > 1e-4]
        self.mean_killpoints = col_killpoints.mean()
        # print('The mean of non-none killPoints is %.3f' % mean_killpoints)
        df.loc[df['killPoints'] < 1e-4, 'killPoints'] = self.mean_killpoints
        
        # Replace None values of winPoints with mean
        col_winpoints = df['winPoints']
        col_winpoints = col_winpoints.loc[col_winpoints > 1e-4]
        self.mean_winpoints = col_winpoints.mean()
        # print('The mean of non-none winPoints is %.3f' % mean_winpoints)
        df.loc[df['winPoints'] < 1e-4, 'winPoints'] = self.mean_winpoints
        
        # Feature engineering
        return self.feature_engineering(df, is_train=True)
    
    def transform_test(self, df):
        return self.transform_test_directly(df, self.mean_rankpoints,
                                            self.mean_killpoints, self.mean_winpoints)
    
    def transform_test_directly(self, df, mean_rankpoints, mean_killpoints, mean_winpoints):
        # Replace None values of rankPoints, killPoints, winPoints
        df.loc[df['rankPoints'] < 1e-4, 'rankPoints'] = mean_rankpoints
        df.loc[df['killPoints'] < 1e-4, 'killPoints'] = mean_killpoints
        df.loc[df['winPoints'] < 1e-4, 'winPoints'] = mean_winpoints
        
        # Feature engineering
        return self.feature_engineering(df, is_train=False)
    
    def transform_prediction(self, df_meta, y_predicted, align=False):
        df_y = df_meta.copy()
        
        if align:
            df_y['winPlacePerc_raw'] = y_predicted
            group_by = df_y.groupby(['matchId'])
            df_y['rank'] = group_by['winPlacePerc_raw'].rank()
            df_y = df_y.merge(
                group_by.size().to_frame('matchSize').reset_index(),
                on='matchId', how='left'
            )
            group_by = None
            df_y = df_y.assign(winPlacePerc=df.apply(
                lambda row: 0.0 if row['matchSize'] == 1 else ((row['matchSize'] - 1) / (row['matchSize'] - 1)),
                axis=1
            ))
        else:
            df_y['winPlacePerc'] = y_predicted
        
        return (self.df_test_meta.merge(df_y[['matchId', 'groupId', 'winPlacePerc']],
                                        on=['matchId', 'groupId'], how='left'))[['Id', 'winPlacePerc']]
    
    def feature_engineering(self, df, is_train=True):
        # Add hand-engineered features
        df_walkDistance = df['walkDistance'] + 5
        df['heals_over_dist'] = df['heals'] / df_walkDistance
        df['boosts_over_dist'] = df['boosts'] / df_walkDistance
        df['kills_over_dist'] = df['kills'] / df_walkDistance
        df['headshots_over_dist'] = df['headshotKills'] / df_walkDistance
        df['killStreaks_over_dist'] = df['killStreaks'] / df_walkDistance
        df['damageDealt_over_dist'] = df['damageDealt'] / df_walkDistance
        df['dbnos_over_dist'] = df['DBNOs'] / df_walkDistance
        df['weapons_over_dist'] = df['weaponsAcquired'] / df_walkDistance
        df['revives_over_dist'] = df['revives'] / df_walkDistance
        df_walkDistance = None
        df_kills = df['kills'] + 0.001
        df['headshots_over_kills'] = df['headshotKills'] / df_kills
        df['killStreaks_over_kills'] = df['killStreaks'] / df_kills
        df_kills = None
        df['teamwork'] = df['assists'] + df['revives']
        df['totalDistance'] = df['walkDistance'] + df['rideDistance'] + df['swimDistance']
        df['items'] = df['heals'] + df['boosts']
        df['skills'] = df['headshotKills'] + df['roadKills']
        
        features = df.columns.tolist()
        features.remove('Id')
        features.remove('groupId')
        features.remove('matchId')
        features.remove('matchType')
        if is_train: features.remove('winPlacePerc')
            
        # Add group mean and group mean rank in match
        group_by = df.groupby(['matchId','groupId'])
        group_by_features = group_by[features]
        df_agg = group_by_features.agg('mean')
        df_agg_rank = df_agg.groupby('matchId')[features].rank(pct=True).reset_index()
        df_out = df_agg.reset_index().merge(df_agg_rank, on=['matchId', 'groupId'], how='left',
                                            suffixes=["_groupMean", "_groupMeanRank"])

        # Add group max and group max rank in match
        df_agg = group_by_features.agg('max')
        df_agg_rank = df_agg.groupby('matchId')[features].rank(pct=True).reset_index()
        df_agg = df_agg.reset_index().merge(df_agg_rank, on=['matchId', 'groupId'], how='left',
                                            suffixes=["_groupMax", "_groupMaxRank"])
        df_out = df_out.merge(df_agg, on=['matchId', 'groupId'], how='left')

        # Add group min and group min rank in match
        df_agg = group_by_features.agg('min')
        df_agg_rank = df_agg.groupby('matchId')[features].rank(pct=True).reset_index()
        df_agg = df_agg.reset_index().merge(df_agg_rank, on=['matchId', 'groupId'], how='left',
                                            suffixes=["_groupMin", "_groupMinRank"])
        df_out = df_out.merge(df_agg, on=['matchId', 'groupId'], how='left')
        
        
        # Add group sum and group sum rank in match
        df_agg = group_by_features.agg('sum')
        df_agg_rank = df_agg.groupby('matchId')[features].rank(pct=True).reset_index()
        df_agg = df_agg.reset_index().merge(df_agg_rank, on=['matchId', 'groupId'], how='left',
                                            suffixes=["_groupSum", "_groupSumRank"])
        df_out = df_out.merge(df_agg, on=['matchId', 'groupId'], how='left')
        group_by_features = None
        
        # Add group size
        df_agg = group_by.size().to_frame('groupSize').reset_index()
        df_out = df_out.merge(df_agg, on=['matchId', 'groupId'], how='left')

        # Add target if for training
        if is_train:
            df_agg = group_by[['winPlacePerc']].first().reset_index()
            df_out = df_out.merge(df_agg, on=['matchId', 'groupId'], how='left')
            
        # Add match mean
        group_by = df.groupby(['matchId'])
        df_agg = group_by[features].agg('mean').reset_index()
        df_out = df_out.merge(df_agg, on=['matchId'], how='left') # Original name now represents match mean
            
        # Add match size
        df_agg = group_by['groupId'].nunique().to_frame('matchSize').reset_index()
        df_out = df_out.merge(df_agg, on=['matchId'], how='left')

        # Add encoded matchType (no improvement)
        # df_agg = group_by['matchType'].first().reset_index()
        # df_out = df_out.merge(df_agg, on=['matchId'], how='left')
        # df_out = pd.get_dummies(df_out, columns=['matchType']) # Ont-hot encoding
        # df_out['matchType'] = df_out['matchType'].astype('category').cat.codes # Label encoding
        
        # Keep metadata of df_test for later restoring individual prediction
        if not is_train:
            self.df_test_meta = df[['Id', 'matchId', 'groupId']]
        
        # print(df_out)
        # Return: features, metadata, weights
        return df_out.drop(columns=['matchId', 'groupId']),\
               df_out[['matchId', 'groupId']],\
               df_out['groupSize'].values

# Unit test for DfPreprocessor
df_bbb = df_train.iloc[:6, :].copy()
df_bbb.iloc[0, 26] = 0
df_bbb.iloc[1:5, 2] = df_bbb.iloc[0, 2]
df_bbb.iloc[1:3, 1] = df_bbb.iloc[0, 1]
df_bbb.iloc[1:3, 28] = df_bbb.iloc[0, 28]
df_bbb, df_bbb_meta, bbb_weights = DfTransformer().transform_train(df_bbb)

In [4]:
# Preprocess data
df_transformer = DfTransformer()
df_train, df_train_meta, weight_train = df_transformer.transform_train(df_train)

In [5]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() 
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() 
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

df_train = reduce_mem_usage(df_train)

Memory usage of dataframe is 4037274048.00 MB
Memory usage after optimization is: 960676656.00 MB
Decreased by 76.2%
Memory usage of dataframe is 99200.00 MB
Memory usage after optimization is: 22000.00 MB
Decreased by 77.8%


In [5]:
# Get X and y
y_train = df_train['winPlacePerc'].values
X_train = df_train.drop(columns='winPlacePerc').values
df_train = None

print(X_train.shape)
gc.collect()

(2026744, 353)


182

In [9]:
# Standardize
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

standardize = False
if standardize:
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

In [6]:
K = 2

In [9]:
# Define model parameters
lgb_params = {"objective" : "regression", "metric" : "mae", 'n_estimators':24000, 'early_stopping_rounds':300,
              "num_leaves" : 40, "learning_rate" : 0.05, "bagging_fraction" : 0.7,
               "bagging_seed" : 0, "num_threads" : 4,"colsample_bytree" : 0.7
             }

In [8]:
# Define method to get k-fold CV MAE
import lightgbm as lgb

def report_mae_lightgbm(lgb_params, lgb_data, K=2):
    cv_metrics = lgb.cv(lgb_params, lgb_data, nfold=K, stratified=False, verbose_eval=1000)
    return cv_metrics['l1-mean'][-1]

In [9]:
# Compute MAE by 2-fold CV
lgb_data = lgb.Dataset(X_train, label=y_train, weight=weight_train, free_raw_data=False)

mae_mean = report_mae_lightgbm(lgb_params, lgb_data, K)
print('LightGBM MAE: %.5f' % mae_mean)

/home/lxu85/playground/lib/python3.5/site-packages/lightgbm/engine.py:426: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/lxu85/playground/lib/python3.5/site-packages/lightgbm/engine.py:431: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


LightGBM MAE: 0.02404


In [7]:
# Prepare validation set
import lightgbm as lgb

ratio_valid = 0.10

idx_shuffle = np.arange(X_train.shape[0])
np.random.shuffle(idx_shuffle)
idx_split = int(X_train.shape[0] * ratio_valid)
idx_valid = idx_shuffle[:idx_split]
idx_train = idx_shuffle[idx_split:]

X_valid = X_train[idx_valid]
y_valid = y_train[idx_valid]
weight_valid = weight_train[idx_valid]
lgb_data_valid = lgb.Dataset(X_valid, label=y_valid, weight=weight_valid, free_raw_data=True)

X_train = X_train[idx_train]
y_train = y_train[idx_train]
weight_train = weight_train[idx_train]
lgb_data_train = lgb.Dataset(X_train, label=y_train, weight=weight_train, free_raw_data=True)

gc.collect()

0

In [10]:
# Train model
model = lgb.train(lgb_params, lgb_data_train,
                  valid_sets=[lgb_data_train, lgb_data_valid],
                  early_stopping_rounds=300,
                  verbose_eval=1000)

/home/lxu85/playground/lib/python3.5/site-packages/lightgbm/engine.py:116: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/lxu85/playground/lib/python3.5/site-packages/lightgbm/engine.py:121: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 300 rounds.
[1000]	training's l1: 0.0254798	valid_1's l1: 0.0258968
[2000]	training's l1: 0.0242812	valid_1's l1: 0.0252021
[3000]	training's l1: 0.0234324	valid_1's l1: 0.0248162
[4000]	training's l1: 0.0227291	valid_1's l1: 0.024535
[5000]	training's l1: 0.0221028	valid_1's l1: 0.0243036
[6000]	training's l1: 0.0215547	valid_1's l1: 0.0241267
[7000]	training's l1: 0.02105	valid_1's l1: 0.0239833
[8000]	training's l1: 0.0205831	valid_1's l1: 0.0238576
[9000]	training's l1: 0.0201375	valid_1's l1: 0.0237371
[10000]	training's l1: 0.0197213	valid_1's l1: 0.0236291
[11000]	training's l1: 0.0193321	valid_1's l1: 0.0235345
[12000]	training's l1: 0.018962	valid_1's l1: 0.0234493
[13000]	training's l1: 0.0185997	valid_1's l1: 0.0233635
[14000]	training's l1: 0.0182519	valid_1's l1: 0.0232842
[15000]	training's l1: 0.0179243	valid_1's l1: 0.0232174
[16000]	training's l1: 0.0176076	valid_1's l1: 0.0231457
[17000]	training's l1: 0.0173031	valid

In [11]:
# Save model
model.save_model('LightGBM_Model.txt')